In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import CustomHyperModel
import CustomWindowGenerator
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

# DATASET

## Load Dataset

In [2]:
data_horaria_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Horaria.xlsx')
data_horaria = pd.read_excel(data_horaria_path)
data_horaria = data_horaria.set_index('Fecha')

In [3]:
precio_bolsa_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [4]:
nombre_series = data_horaria.columns

In [5]:
full_df = pd.concat([data_horaria,precio_bolsa],axis=1)
full_df.shape,data_horaria.shape,precio_bolsa.shape

((176760, 85), (176760, 84), (176760, 1))

## Build Window

In [6]:
d = 'All'
IndLastMonth = '2020-01-01'
n_steps_in = 48
n_steps_out=24
overlap = 24
inputs_columns = nombre_series
output_columns = ['$kWh']

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

trainX, trainY, testX, testY, scaler_x, scaler_y, df2, dataset = Ep.SplitTimeseriesMultipleTimesBackAhead(full_df,
                                                                                              day = d, 
                                                                                              ValData = 'index', 
                                                                                              TimeAhead = IndLastMonth, 
                                                                                              n_steps_out= n_steps_out, 
                                                                                              n_steps_in = n_steps_in, 
                                                                                              overlap = overlap,
                                                                                              input_features=inputs_columns,
                                                                                              output_features=output_columns)
trainX.shape,trainY.shape,testX.shape,testY.shape

((7272, 48, 84), (7272, 24, 1), (91, 48, 84), (91, 24, 1))

In [7]:
"""
log_dir = os.path.join('logs','Optimización')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
callback_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                          histogram_freq=2,
                                                          write_graph=False,
                                                          update_freq='epoch')
"""

callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          factor=0.1,
                                                          min_lr=1e-4,
                                                          patience=0,
                                                          verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=5,
                                                  mode='min')

#callbacks = [callback_tensorboard,callback_reduce_lr,early_stopping]
callbacks = [callback_reduce_lr,early_stopping]

In [8]:
INPUT_SHAPE = (n_steps_in,len_input_features)

arquitectura16 = CustomHyperModel.Arquitectura16(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)
arquitectura17 = CustomHyperModel.Arquitectura17(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)
arquitectura18 = CustomHyperModel.Arquitectura18(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)
arquitectura19 = CustomHyperModel.Arquitectura19(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)
arquitectura20 = CustomHyperModel.Arquitectura20(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)
arquitectura21 = CustomHyperModel.Arquitectura21(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)
arquitectura22 = CustomHyperModel.Arquitectura22(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)
arquitectura23 = CustomHyperModel.Arquitectura23(input_shape=INPUT_SHAPE,output_units=len_output_features,n_steps_out=n_steps_out)

In [25]:
bayesian_tuner = BayesianOptimization(
    arquitectura23,
    objective='val_mean_absolute_percentage_error',
    num_initial_points=1,
    max_trials=15,
    directory='my_dir',
    project_name='tuning_23'
)

# Overview of the task
bayesian_tuner.search_space_summary()

# Performs the hyperparameter tuning
search_start = time.time()
bayesian_tuner.search(x=trainX,y=trainY,
                      epochs=200,
                      validation_data=(testX,testY),
                      callbacks=callbacks)
search_end = time.time()
elapsed_time = search_end - search_start

Trial 15 Complete [00h 00m 24s]
val_mean_absolute_percentage_error: 32.68419647216797

Best val_mean_absolute_percentage_error So Far: 22.615131378173828
Total elapsed time: 00h 10m 20s
INFO:tensorflow:Oracle triggered exit


In [26]:
print('Tiempo Total Transcurrido {}'.format(elapsed_time))

# Show a summary of the search
bayesian_tuner.results_summary()

Tiempo Total Transcurrido 620.8955624103546
Results summary
Results in my_dir\tuning_23
Showing 10 best trials
Objective(name='val_mean_absolute_percentage_error', direction='min')
Trial summary
Hyperparameters:
gru_units_layer_1: 448
kernel_regularizer_layer_1: 0.105
dropout_regularizer_layer_1: 0.0
learning_rate: 0.01
Score: 22.615131378173828
Trial summary
Hyperparameters:
gru_units_layer_1: 256
kernel_regularizer_layer_1: 0.0675
dropout_regularizer_layer_1: 0.8099999999999999
learning_rate: 0.004485708249434565
Score: 23.492027282714844
Trial summary
Hyperparameters:
gru_units_layer_1: 64
kernel_regularizer_layer_1: 0.06
dropout_regularizer_layer_1: 0.0
learning_rate: 0.01
Score: 24.703332901000977
Trial summary
Hyperparameters:
gru_units_layer_1: 64
kernel_regularizer_layer_1: 0.105
dropout_regularizer_layer_1: 0.99
learning_rate: 0.01
Score: 25.533926010131836
Trial summary
Hyperparameters:
gru_units_layer_1: 512
kernel_regularizer_layer_1: 0.105
dropout_regularizer_layer_1: 0.99

In [11]:
# Retrieve the best model.
best_model = bayesian_tuner.get_best_models(num_models=1)[0]

# Evaluate the best model.
loss, mae, mape, smape = best_model.evaluate(x=testX,y=testY)

3/3 [==============================] - 0s 10ms/step - loss: 43.9824 - mean_absolute_error: 0.0527 - mean_absolute_percentage_error: 41.9580 - symmetric_mean_absolute_percentage_error: 1224.8458


In [12]:
best_model.to_json()

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 48, 84], "dtype": "float32", "sparse": false, "ragged": false, "name": "lstm_input"}}, {"class_name": "LSTM", "config": {"name": "lstm", "trainable": true, "batch_input_shape": [null, 48, 84], "dtype": "float32", "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "time_major": false, "units": 320, "activation": "tanh", "recurrent_activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "recurrent_initializer": {"class_name": "Orthogonal", "config": {"gain": 1.0, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "unit_forget_bias": true, "kernel_regularizer": {"class_name": "L1", "config": {"l1": 0.04500000178813934}}, "recurrent_regularizer": null, "bias_regularizer": null, "activity_regularizer": nul